In [71]:
import cirq
import numpy as np
from sklearn.preprocessing import normalize
import sympy as sp
from cirq.contrib.svg import SVGCircuit
import tensorflow_quantum as tfq
from tensorflow_quantum.python import util
import re

In [2]:
def beta(s, j, x):
    index_num = (2*j-1)*(2**(s-1))
    index_den = (j-1)*(2**s)
    
    num = np.sqrt(np.sum(abs(x[index_num : index_num+2**(s-1)])**2))
    den = np.sqrt(np.sum(abs(x[index_den : index_den+2**(s)])**2))
    
    if den == 0:
        beta = 0
    else:
        beta = 2*np.arcsin(num/den)
    print(beta)
    return beta

In [3]:
def locate_x(curr_j, prev_j, length):
    curr_bin = bin(curr_j)[2:].zfill(length)
    prev_bin = bin(prev_j)[2:].zfill(length)
    return [i for i, (x,y) in enumerate(zip(curr_bin,prev_bin)) if x!=y]

In [4]:
def amplitude_embedding(x):
    n = int(np.log2(len(x)))
    qubits = cirq.GridQubit.rect(1, n)
    circuit = cirq.Circuit()
    
    circuit += cirq.ry(beta(n, 1, x))(qubits[0])
    
    for i in range(1,n):
        # We can have at most i control bits
        # Total possibilities is therefore 2^i
        controls = 2**i
        
        control_qubits = [qubits[c] for c in range(i+1)]
        circuit += cirq.ControlledGate(sub_gate=cirq.ry(beta(n-i, controls, x)), 
                                       num_controls=len(control_qubits)-1)(*control_qubits)
        
        for j in range(1, controls):
            for loc in locate_x(controls-j-1, controls-j, i):
                circuit += cirq.X(qubits[loc])
                
            circuit += cirq.ControlledGate(sub_gate=cirq.ry(beta(n-i, controls-j, x)), 
                                       num_controls=len(control_qubits)-1)(*control_qubits)
            
        for k in range(i):
            circuit += cirq.X(qubits[k])

    return circuit

In [5]:
x = np.arange(1,5,1)
d = np.sqrt(np.sum(np.square(x)))
x = x/d
print('amplitudes:',x)
print('probs:',np.square(x))
qubits = cirq.GridQubit.rect(1,int(np.ceil(np.log2(len(x)))))
qc = cirq.Circuit()
qc += amplitude_embedding(x)
qc += cirq.measure(*qubits, key='result')

amplitudes: [0.18257419 0.36514837 0.54772256 0.73029674]
probs: [0.03333333 0.13333333 0.3        0.53333333]
2.300523983021863
1.8545904360032246
2.2142974355881813


In [6]:
x = np.arange(5,9,1)
d = np.sqrt(np.sum(np.square(x)))
x = x/d
print('amplitudes:',x)
print('probs:',np.square(x))
qubits = cirq.GridQubit.rect(1,int(np.ceil(np.log2(len(x)))))
qc = cirq.Circuit()
qc += amplitude_embedding(x)
qc += cirq.measure(*qubits, key='result')

amplitudes: [0.37904902 0.45485883 0.53066863 0.60647843]
probs: [0.14367816 0.20689655 0.2816092  0.36781609]
1.8742842836248206
1.7039326543465443
1.7521161011963868


In [7]:
qc

(0, 0): ───Ry(0.597π)───@────────────X───@────────────X───M('result')───
                        │                │                │
(0, 1): ────────────────Ry(0.542π)───────Ry(0.558π)───────M─────────────

In [8]:
s=cirq.Simulator()
shots = 10000
samples=s.run(qc, repetitions=shots)
res = dict(samples.histogram(key="result"))
for key, value in res.items():
    res[key] = value/shots
res

{1: 0.2041, 2: 0.2795, 3: 0.3706, 0: 0.1458}

## Amplitude Encoder

In [9]:
def _locate_x(curr_j, prev_j, length):
    curr_bin = bin(curr_j)[2:].zfill(length)
    prev_bin = bin(prev_j)[2:].zfill(length)
    return [i for i, (x, y) in enumerate(zip(curr_bin, prev_bin)) if x != y]

def build(qubits, symbols):
    n = len(qubits)
    ae_ops = []
    count = 0
    ae_ops += [cirq.ry(symbols[count])(qubits[0])]
    count += 1
    for i in range(1, n):
        # We can have at most i control bits
        # Total possibilities is therefore 2^i
        controls = 2**i

        control_qubits = [qubits[c] for c in range(i + 1)]
        ae_ops += [
            cirq.ControlledGate(sub_gate=cirq.ry(symbols[count]),
                                num_controls=len(control_qubits) -
                                1)(*control_qubits)
        ]
        count += 1
        for j in range(1, controls):
            for loc in _locate_x(controls - j - 1, controls - j, i):
                ae_ops += [cirq.X(qubits[loc])]

            ae_ops += [
                cirq.ControlledGate(sub_gate=cirq.ry(symbols[count]),
                                    num_controls=len(control_qubits) -
                                    1)(*control_qubits)
            ]
            count += 1

        for k in range(i):
            ae_ops += [cirq.X(qubits[k])]

    return ae_ops

In [10]:
def _beta_range(s,j):
    index_num = (2 * j - 1) * (2**(s - 1))
    index_den = (j - 1) * (2**s)

    num_start = index_num
    num_end = index_num + 2**(s - 1)

    den_start = index_den
    den_end = index_den + 2**(s)
    
    return [[num_start,num_end], [den_start,den_end]]

def amplitude_encoder(x):
    print("Amplitude encoder...")
#         img_size = self.dims
#         x = x.reshape(-1, np.prod(img_size))
#         len = x.shape[1]
#         n_qubits = int(np.ceil(np.log2(len)))
#         max_len = 2**n_qubits
#         x = pad_sequences(x, maxlen=max_len, padding='post')

#         n = n_qubits
    n = n = int(np.log2(x.shape[1]))
#     slices = [] # array of [s,j]
#     slices.append([n,1])

#     for i in range(1, n):
#         controls = 2**i
#         slices.append([n-i, controls])

#         for j in range(1, controls):
#             slices.append([n-i,controls-j])
            
    s = np.array([ _beta_range(n-i,2**i - j) for i in range(n) for j in range(2**i) ])
    indices = np.arange(x.shape[1])
    mask_num = (indices < s[:,0,:][:,0][:,None]) | (indices >= s[:,0,:][:,1][:,None])
    mask_den = (indices < s[:,1,:][:,0][:,None]) | (indices >= s[:,1,:][:,1][:,None])
    
    
    as_strided = np.lib.stride_tricks.as_strided
    
    strided = np.array([as_strided(x[i], shape= mask_num.shape, strides = (0, x.strides[1]))  for i in range(x.shape[0])])
    mask_num = np.repeat(mask_num[None,...],x.shape[0],axis=0)
    mask_den = np.repeat(mask_den[None,...],x.shape[0],axis=0)
    
    num_coeff = np.ma.sqrt(np.ma.sum(np.ma.power(np.ma.array(strided, mask=mask_num).__abs__(),2),axis = 2))
    den_coeff = np.ma.sqrt(np.ma.sum(np.ma.power(np.ma.array(strided, mask=mask_den).__abs__(), 2), axis = 2))
    
    res = 2*np.arcsin(num_coeff/den_coeff)
    
    return res

In [11]:
x = [[1,2,3,4]]
d = np.sqrt(np.sum(np.square(x), axis = 1))
x = x/d[:,None]
print('amplitudes:\n',x)
print('probs:\n',np.square(x))
res = amplitude_encoder(x)
print(res)

qubits = cirq.GridQubit.rect(1,int(np.ceil(np.log2(x.shape[1]))))
qc = cirq.Circuit()

in_symbols = sp.symbols(f'x0:{res[0].shape[0]}')
in_symbols = np.asarray(in_symbols).reshape((res[0].shape[0]))

qc += build(qubits,in_symbols)
qc += cirq.measure(*qubits, key='result')

amplitudes:
 [[0.18257419 0.36514837 0.54772256 0.73029674]]
probs:
 [[0.03333333 0.13333333 0.3        0.53333333]]
Amplitude encoder...
[[2.300523983021863 1.8545904360032246 2.2142974355881813]]


In [12]:
qc

(0, 0): ───Ry(x0)───@────────X───@────────X───M('result')───
                    │            │            │
(0, 1): ────────────Ry(x1)───────Ry(x2)───────M─────────────

In [13]:
rqc = cirq.resolve_parameters(qc, { j: res[0][i] for i,j in enumerate(in_symbols)})
rqc

(0, 0): ───Ry(0.732π)───@───────────X───@────────────X───M('result')───
                        │               │                │
(0, 1): ────────────────Ry(0.59π)───────Ry(0.705π)───────M─────────────

In [14]:
s=cirq.Simulator()
shots = 10000
samples=s.run(rqc, repetitions=shots)
out = dict(samples.histogram(key="result"))
for key, value in out.items():
    out[key] = value/shots
out

{3: 0.537, 2: 0.2981, 1: 0.1334, 0: 0.0315}

In [15]:
qubits = cirq.GridQubit.rect(1,int(np.ceil(np.log2(x.shape[1]))))
qc = cirq.Circuit()

in_symbols = sp.symbols(f'x0:{res[1].shape[0]}')
in_symbols = np.asarray(in_symbols).reshape((res[1].shape[0]))

qc += build(qubits,in_symbols)
qc += cirq.measure(*qubits, key='result')

IndexError: index 1 is out of bounds for axis 0 with size 1

In [16]:
qc

In [17]:
rqc = cirq.resolve_parameters(qc, { j: res[1][i] for i,j in enumerate(in_symbols)})
rqc

IndexError: index 1 is out of bounds for axis 0 with size 1

In [18]:
s=cirq.Simulator()
shots = 10000
samples=s.run(rqc, repetitions=shots)
out = dict(samples.histogram(key="result"))
for key, value in out.items():
    out[key] = value/shots
out

{3: 0.5394, 2: 0.2949, 1: 0.1321, 0: 0.0336}

In [62]:
qubits = cirq.GridQubit.rect(1,3)
qc = cirq.Circuit()
x = sp.symbols('x0:3')
qc.append(cirq.rx(x[0]*x[1])(qubits[0]))
qc.append(cirq.rx(x[1]*x[2])(qubits[1]))
qc.append(cirq.rx(x[2]*x[0])(qubits[2]))

In [63]:
qc

(0, 0): ───Rx(x0*x1)───

(0, 1): ───Rx(x1*x2)───

(0, 2): ───Rx(x0*x2)───

In [64]:
def get_circuit_qubits(circuit):
    """Returns a list of qubits in a circuit
    
    Arguments:
        circuit: cirq.Circuit, quple.QuantumCircuit
            The circuit to find the associated qubits
    Returns:
        A list of qubits in the circuit
    """      
    all_qubits = set()
    for moment in circuit:
        for op in moment:
            all_qubits |= set(op._qubits)
    return sorted(list(all_qubits))


def natural_key(symbol):
    '''Keys for human sorting
    Reference:
    http://nedbatchelder.com/blog/200712/human_sorting.html
    '''
    return [atoi(s) for s in re.split(r'(\d+)', symbol.name)]


def get_circuit_symbols(circuit, to_str=True, sort_key=natural_key):
    """Returns a list of parameter symbols in a circuit
    
    Arguments:
        circuit: cirq.Circuit, quple.QuantumCircuit
            The circuit to find the associated parameter symbols
        to_str: boolean, default=True
            Whether to convert symbol to strings
        sort_key:
            Sort key for the list of symbols
    Returns:
        A list of symbols in the circuit
    """
    all_symbols = set()
    for moment in circuit:
        for op in moment:
            if cirq.is_parameterized(op):
                all_symbols |= symbols_in_op(op.gate)
    sorted_symbols = sorted(list(all_symbols), key=sort_key)
    if to_str:
        return [str(x) for x in sorted_symbols]
    return sorted_symbols

def symbols_in_op(op):
    """Returns the set of symbols associated with a parameterized gate operation.
    
    Arguments:
        op: cirq.Gate
            The parameterised gate operation to find the set of symbols associated with
    
    Returns:
        Set of symbols associated with the parameterized gate operation
    """
    if isinstance(op, cirq.EigenGate):
        return op.exponent.free_symbols

    if isinstance(op, cirq.FSimGate):
        ret = set()
        if isinstance(op.theta, sympy.Basic):
            ret |= op.theta.free_symbols
        if isinstance(op.phi, sympy.Basic):
            ret |= op.phi.free_symbols
        return ret

    if isinstance(op, cirq.PhasedXPowGate):
        ret = set()
        if isinstance(op.exponent, sympy.Basic):
            ret |= op.exponent.free_symbols
        if isinstance(op.phase_exponent, sympy.Basic):
            ret |= op.phase_exponent.free_symbols
        return ret

    raise ValueError("Attempted to scan for symbols in circuit with unsupported"
                     " ops inside. Expected op found in tfq.get_supported_gates"
                     " but found: ".format(str(op)))

    
def atoi(symbol):
    return int(symbol) if symbol.isdigit() else symbol

In [82]:
type(cirq.flatten(qc)[0])

cirq.circuits.circuit.Circuit

In [94]:
cq , emap = cirq.flatten(qc)

In [95]:
cq

(0, 0): ───Rx(pi*<x0*x1/pi>)───

(0, 1): ───Rx(pi*<x1*x2/pi>)───

(0, 2): ───Rx(pi*<x0*x2/pi>)───

In [96]:
emap

cirq.ExpressionMap({x0*x1/pi: <x0*x1/pi>, x1*x2/pi: <x1*x2/pi>, x0*x2/pi: <x0*x2/pi>})

In [97]:
util.convert_to_tensor([cq])

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'\n\x0e\n\x0ctfq_gate_set\x12\xb2\x03\x08\x01\x12\xad\x03\n\x8c\x01\n\x04\n\x02XP\x12\x16\n\x0econtrol_values\x12\x04\n\x02\x1a\x00\x12\x16\n\x0econtrol_qubits\x12\x04\n\x02\x1a\x00\x12\x1a\n\x0fexponent_scalar\x12\x07\n\x05\r\x00\x00\x80?\x12\x18\n\x08exponent\x12\x0c\x12\n<x0*x1/pi>\x12\x17\n\x0cglobal_shift\x12\x07\n\x05\r\x00\x00\x00\xbf\x1a\x05\x12\x030_0\n\x8c\x01\n\x04\n\x02XP\x12\x16\n\x0econtrol_qubits\x12\x04\n\x02\x1a\x00\x12\x16\n\x0econtrol_values\x12\x04\n\x02\x1a\x00\x12\x18\n\x08exponent\x12\x0c\x12\n<x1*x2/pi>\x12\x1a\n\x0fexponent_scalar\x12\x07\n\x05\r\x00\x00\x80?\x12\x17\n\x0cglobal_shift\x12\x07\n\x05\r\x00\x00\x00\xbf\x1a\x05\x12\x030_1\n\x8c\x01\n\x04\n\x02XP\x12\x17\n\x0cglobal_shift\x12\x07\n\x05\r\x00\x00\x00\xbf\x12\x16\n\x0econtrol_qubits\x12\x04\n\x02\x1a\x00\x12\x1a\n\x0fexponent_scalar\x12\x07\n\x05\r\x00\x00\x80?\x12\x18\n\x08exponent\x12\x0c\x12\n<x0*x2/pi>\x12\x16\n\x0econtrol_values\x12\x04\n\x02\x

In [100]:
rqc = cirq.resolve_parameters(cq, { j: 1 for i,j in enumerate(x)})
rqc += cirq.measure(*qubits, key='result')
rqc

(0, 0): ───Rx(pi*<x0*x1/pi>)───M('result')───
                               │
(0, 1): ───Rx(pi*<x1*x2/pi>)───M─────────────
                               │
(0, 2): ───Rx(pi*<x0*x2/pi>)───M─────────────

In [101]:
s=cirq.Simulator()
shots = 10000
samples=s.run(rqc, repetitions=shots)
out = dict(samples.histogram(key="result"))
for key, value in out.items():
    out[key] = value/shots
out

ValueError: Circuit contains ops whose symbols were not specified in parameter sweep. Ops: [cirq.Rx(rads=sympy.Mul(sympy.pi, sympy.Symbol('<x0*x1/pi>'))).on(cirq.GridQubit(0, 0)), cirq.Rx(rads=sympy.Mul(sympy.pi, sympy.Symbol('<x1*x2/pi>'))).on(cirq.GridQubit(0, 1)), cirq.Rx(rads=sympy.Mul(sympy.pi, sympy.Symbol('<x0*x2/pi>'))).on(cirq.GridQubit(0, 2))]

In [102]:
cq

(0, 0): ───Rx(pi*<x0*x1/pi>)───

(0, 1): ───Rx(pi*<x1*x2/pi>)───

(0, 2): ───Rx(pi*<x0*x2/pi>)───

In [103]:
qc

(0, 0): ───Rx(x0*x1)───

(0, 1): ───Rx(x1*x2)───

(0, 2): ───Rx(x0*x2)───

In [104]:
cq + qc

(0, 0): ───Rx(pi*<x0*x1/pi>)───Rx(x0*x1)───

(0, 1): ───Rx(pi*<x1*x2/pi>)───Rx(x1*x2)───

(0, 2): ───Rx(pi*<x0*x2/pi>)───Rx(x0*x2)───

In [111]:
sp.Abs(x[0])

Abs(x0)

In [113]:
qc.

[cirq.Moment(
     cirq.Rx(rads=sympy.Mul(sympy.Symbol('x0'), sympy.Symbol('x1'))).on(cirq.GridQubit(0, 0)),
     cirq.Rx(rads=sympy.Mul(sympy.Symbol('x1'), sympy.Symbol('x2'))).on(cirq.GridQubit(0, 1)),
     cirq.Rx(rads=sympy.Mul(sympy.Symbol('x0'), sympy.Symbol('x2'))).on(cirq.GridQubit(0, 2)),
 )]